In [1]:
import os
from Bio import SeqIO
from Bio.Seq import MutableSeq
from Bio.Seq import Seq
import re
import csv
import pandas as pd 

find genes wthin sites and claculate amino change

In [2]:


fp="/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/nodups_cov5_HE_PLUS_STAR100len/Edits_only/VEP_out.csv"
vep = pd.read_csv(filepath_or_buffer=fp, sep="\t")
#fix location column
vep[["chr","Location",]]=vep["Location"].str.split(pat=":",expand=True,n=-1)
vep[["garb","Location",]]=vep["Location"].str.split(pat="-",expand=True,n=-1)
vep.drop(['garb'],axis=1)
#fix type
vep["Location"] = vep["Location"].astype('int')
# remove not refseq-curated sites 
vep =vep[vep['Feature'].str.startswith('NM')]
# remove none cds sites
vep =vep[vep['CDS_position'] != '-']


# match edit sites to genes
edit_site_path = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/nodups_cov5_HE_PLUS_STAR100len/Edits_only/all_uniq_by_frequncy_sites.known.tab"
# list for all sites with their change
list_sites_with_genes = []
with open(edit_site_path,"r") as geneFd:
    DictReader_obj = csv.DictReader(geneFd, delimiter="\t")
    for site in DictReader_obj:
        site['Coverage-q25'] = int(float(site['Coverage-q25']))
        site_vep = vep[vep["Location"] == int(site["Position"])]
        # row for every gene symbol 
        for gene_symb in site_vep['SYMBOL'].unique():
            symb_site_vep = site_vep[site_vep["SYMBOL"] == gene_symb]
            # will save all the site data even after closing the file
            site_with_changes = site.copy()
            site_with_changes['mut_type'] = []
            # for the site's genes acc num
            site_with_changes['genes'] = []
            # for the site's amino change per gene
            site_with_changes['aminos_change'] =[]
            site_with_changes['gene_name']=symb_site_vep.loc[symb_site_vep.index[0],'SYMBOL']
            # iterate rows - adding results for every isoform to the symbol row
            for index, gene in symb_site_vep.iterrows():
                site_with_changes['genes'].append(gene['Feature'])
                site_with_changes['mut_type'].append(gene['Consequence'])
                if '/' in gene['Amino_acids']:
                    site_with_changes['aminos_change'].append(gene['Amino_acids'].replace('/', gene['Protein_position']))
                else:
                    site_with_changes['aminos_change'].append(gene['Amino_acids']+ gene['Protein_position']+gene['Amino_acids'])
            list_sites_with_genes.append(site_with_changes)
        


In [3]:
out_file = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/nodups_cov5_HE_PLUS_STAR100len/comfort_ver_final_table_amino_change_by_site_byFreq.tab.csv"
with open(out_file, "w") as OFD:
    writer = csv.DictWriter(OFD, delimiter="\t",fieldnames=list(list_sites_with_genes[0].keys()))
    writer.writeheader()
    writer.writerows(list_sites_with_genes) 


In [4]:
for site in list_sites_with_genes:
    site['mut_type']=";".join(site['mut_type'])
    site['aminos_change']=";".join(site['aminos_change'])
    site['genes']=";".join(site['genes'])

In [5]:
out_file = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/final_table_amino_change_by_site_byFreq.tab.csv"
with open(out_file, "w") as OFD:
    writer = csv.DictWriter(OFD, delimiter="\t",fieldnames=list(list_sites_with_genes[0].keys()))
    writer.writeheader()
    writer.writerows(list_sites_with_genes) 


In [6]:

out_file_csv = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/final_table_amino_change_by_site_byFreq.csv"
with open(out_file_csv, "w") as OFD:
    writer = csv.DictWriter(OFD, delimiter=",",fieldnames=list(list_sites_with_genes[0].keys()))
    writer.writeheader()
    writer.writerows(list_sites_with_genes)

In [4]:
# out_file = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/final_table_one_gene_amino_change_by_site_byFreq.tab.csv"
# one_gene_list = []
# for site in list_sites_with_genes:
#     site_copy = site.copy()
#     gene = site_copy['genes'][0]
#     mut_t = site_copy['mut_type'][0]
#     am_c = site_copy['aminos_change'][0]
#     for i in range(len(site['genes'])):
#         if site['mut_type'][i] != 'synonymous_variant':
#             gene = site_copy['genes'][i]
#             mut_t = site_copy['mut_type'][i]
#             am_c = site_copy['aminos_change'][i]
#             break
#     site_copy['genes'] = gene
#     site_copy['mut_type'] = mut_t
#     site_copy['aminos_change'] = am_c
#     one_gene_list.append(site_copy)
#     site['mut_type']=";".join(site['mut_type'])
#     site['aminos_change']=";".join(site['aminos_change'])
#     site['genes']=";".join(site['genes'])
# with open(out_file, "w") as OFD:
#     writer = csv.DictWriter(OFD, delimiter="\t",fieldnames=list(one_gene_list[0].keys()))
#     writer.writeheader()
#     writer.writerows(one_gene_list) 

write results to file

ALL THE SLASHED CODE BELOW IS NICE CODE BUT ITS NEED GENES' SEQ TABLE WITH UPPERCASE CDS (TABLE BROWSER DIDNT WORK FOR THAT)

In [ ]:
# type_of_modifications=['AC','AG','AT','CA','CG','CT','GA','GC','GT','TA','TC','TG']
# standard_genetic_code = {
#     'AUA': 'I', 'AUC': 'I', 'AUU': 'I', 'AUG': 'M',
#     'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACU': 'T',
#     'AAC': 'N', 'AAU': 'N', 'AAA': 'K', 'AAG': 'K',
#     'AGC': 'S', 'AGU': 'S', 'AGA': 'R', 'AGG': 'R',
#     'CUA': 'L', 'CUC': 'L', 'CUG': 'L', 'CUU': 'L',
#     'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCU': 'P',
#     'CAC': 'H', 'CAU': 'H', 'CAA': 'Q', 'CAG': 'Q',
#     'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGU': 'R',
#     'GUA': 'V', 'GUC': 'V', 'GUG': 'V', 'GUU': 'V',
#     'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCU': 'A',
#     'GAC': 'D', 'GAU': 'D', 'GAA': 'E', 'GAG': 'E',
#     'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGU': 'G',
#     'UCA': 'S', 'UCC': 'S', 'UCG': 'S', 'UCU': 'S',
#     'UUC': 'F', 'UUU': 'F', 'UUA': 'L', 'UUG': 'L',
#     'UAC': 'Y', 'UAU': 'Y', 'UAA': None, 'UAG': None,
#     'UGC': 'C', 'UGU': 'C', 'UGA': None, 'UGG': 'W'}

create gene map

In [ ]:
# # read genes seq data
# genes_seq_path = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/znf_seq_all_curated.txt"
# all_genes = {}
# # create map of genes by refseq gen name(acc num)
# with open(genes_seq_path,"r") as genesFD:
#     for record in SeqIO.parse(genesFD, "fasta"):
#         temp = record.description.split(" ")
#         # extract name in two steps to ensure success
#         name = (temp[0]).split("ncbiRefSeqCurated_")
#         assert len(name) > 1, ("error name pasring :"+ str(record.description))
#         name = name[-1]
#         t = temp[1].split(":")
#         region =t[0].split("=")[1]
#         start = t[1].split("-")[0]
#         end = t[1].split("-")[1]
#         strand = temp[4].split("=")[1]
#         seq = str(record.seq)
#         all_genes[name] = {"Name" : name,"Region" : region,  "Start" : int(start),"End" : int(end),"Strand" : strand,"Seq" : seq}
        


COUNT OVERLAPPING GENES ON OPPOSITE STRANDS

In [ ]:

# genes_list = [gene for gene in all_genes.values()]
# counter = 0
# old1=""
# old2=""
# for i in range(len(genes_list)):
#     for j in range(i, len(genes_list)):
#         # for two different genes
#         if i != j:
#             gene1 = genes_list[i]
#             gene2 = genes_list[j]
#             # if they are overllaping
#             if (gene1["Start"] <= gene2["End"] and gene1["End"] >= gene2["End"]) or (gene1["Start"] <= gene2["Start"] and gene1["End"] >= gene2["Start"]):
#                 # if they are on opposite strands
#                 if gene1["Strand"] != gene2["Strand"]:
#                     #if both genes are new
#                     if gene1['Name'] != old1 and gene2['Name'] != old2:
#                         old1=gene1['Name']
#                         old2=gene2['Name']
#                         counter+=1
# print("overlapping events: ", counter)

find duplicated points in REDI input points

In [ ]:
# !!!!!!!!! very long running time - I found no points
# Mf = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/input/Final_file_All_ZNF_A_RefseqCur_knownFormat_sorted.bed"
# with open(Mf, 'r') as FD:
#     lines_splited=[line.split("\t") for line in FD.readlines()]
#     for i in range(len(lines_splited)):
#         for j in range(i+1,len(lines_splited)):
#             if lines_splited[i][0] == lines_splited[j][0] and lines_splited[i][1] == lines_splited[j][1]:
#                 print (lines_splited[i], lines_splited[j], "\n")

In [ ]:

# # match edit sites to genes
# edit_site_path = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/nodups_cov5_HE_PLUS_STAR100len/Edits_only/debug_sites.known.tab"
# # list for all sites with their change
# list_sites_with_genes = []
# with open(edit_site_path,"r") as geneFd:
#     DictReader_obj = csv.DictReader(geneFd, delimiter="\t")
#     for site in DictReader_obj:
#         site['Coverage-q25'] = int(float(site['Coverage-q25']))
#         site['contain_nonsyn'] = 0
#         # for the site's genes id
#         site_genes = []
#         # for the site's amono change per gene
#         site_aminos_change =[]
#         # will save all the site data even after closing the file
#         site_with_changes = site.copy()
#         #find genes wthin the site
#         for name,gene in all_genes.items():
#             # if we find a gene
#             if site["Region"] == gene["Region"] and gene["Start"] < int(site["Position"]) < gene["End"]:
#                 # calc relative position according to strand
#                 gene_seq=gene["Seq"]
#                 print(gene_seq)
#                 # !note! genes seq is already in reverse complement
#                 # !note! that reverse complement will convert CCTGTTC to GAACAGG (complement + reverse) - so the different in strands is only calculation of starting point
#                 if site["Strand"] == "1": # strand is +
#                     gene_relative_site = int(site["Position"]) - gene["Start"]
#                     # make sure editing and gene on the same strand
#                     if gene["Strand"] != "+":
#                         continue   
#                 else:
#                     gene_relative_site = gene["End"] - int(site["Position"])
#                     # make sure editing and gene on the same strand
#                     if gene["Strand"] != "-":
#                         continue   
#                 # make sure site is wthin codin region
#                 if gene_seq[gene_relative_site].islower():
#                     continue
                                 
#                 assert gene_seq[gene_relative_site] == "A", "no A," + str(site) + "gene :"+ name
                
#                 ## for case of other replacement
#                 ## should be removed in the "collect recurrent...ipynb" file
#                 # if gene_seq[gene_relative_site] != "A":
#                 #     assert "AG" not in site["AllSubs"], "AG in subs but not in index " + gene["Seq"] + "strand: " + gene["Strand"]+site["Strand"]+" got "+gene_seq[gene_relative_site] + "area :" + gene_seq[gene_relative_site-3:gene_relative_site+4] +" site: " +str(site)
#                 #     continue
                
#                 #update sitte genes       
#                 site_genes.append(name)
#                 site_with_changes["Genes"] = site_genes
#                 #->find amino replacemant<-
#                 # will point where we are in the list 
#                 idx = 0
#                 # will cout how many aminos so far
#                 amino_counter =1 
#                 border = gene_relative_site+1
                
#                 #find amino change seconded versions
#                 #find where our condon ended
#                 temp_seq = gene_seq[:gene_relative_site+1]
#                 upps = len(re.findall(r'[A-Z]',temp_seq))
#                 mdl=upps%3
#                 if mdl==0:
#                     adding=0
#                 else:
#                     adding=3-mdl
#                 relvant_seq=gene_seq[:gene_relative_site+adding+1]
#                 upps=len(re.findall(r'[A-Z]',relvant_seq))
#                 assert upps%3==0, "%3!=0"
#                 amino_counter=int(upps/3)
#                 # take last 3 chars
#                 seq_relvant_seq=MutableSeq(relvant_seq)
#                 codon=seq_relvant_seq[-3:]
#                 assert str(codon).isupper(), "CODON not upper"+codon
#                 orig_amino=str(codon.translate())
#                 seq_relvant_seq[gene_relative_site]="G"
#                 new_amino=str(seq_relvant_seq[-3:].translate())
#                 if orig_amino != new_amino:
#                             site_with_changes['contain_nonsyn'] = 1
#                 site_aminos_change.append(orig_amino+str(amino_counter)+new_amino)
#                 site_with_changes["Amino_change"] = site_aminos_change
                
                
#                 # while(idx < border):       
#                 #         # if the site is in the current 3 pos
#                 #     if idx+3 > gene_relative_site:
#                 #         # move forward untill its upper 
#                 #         while gene_seq[idx].islower():
#                 #             idx+=1
#                 #         # check we didnt pass the site
#                 #         assert idx <= gene_relative_site, "idx > gene_relative_site"
#                 #         codon = gene_seq[idx:idx+3]
#                 #         # check its three coding nucleotide
#                 #         assert codon[idx:idx+3].isupper, "codon contain lower-cases" + str(codon)
#                 #         # check again its A - otherwise its error in this indexses part
#                 #         assert codon[gene_relative_site - idx] == "A", "error codon no A in place "+str(codon)+ " - " +gene_seq[gene_relative_site-2:gene_relative_site+1]+" > site " + str(gene_relative_site) + " idx "+  str(idx)
#                 #         codon = Seq(codon)
#                 #         orig_amino = str(codon.translate())
#                 #         new_seq = MutableSeq(codon)
#                 #         new_seq[gene_relative_site - idx] = "G"
                        
#                 #         new_amino = str(Seq(new_seq).translate())
#                 #         if orig_amino != new_amino:
#                 #             site_with_changes['contain_nonsyn'] = 1
#                 #         site_aminos_change.append(orig_amino+str(amino_counter)+new_amino)

#                 #         site_with_changes["Amino_change"] = site_aminos_change
#                 #         #finish runing on the seq
#                 #         break
#                 #     elif gene_seq[idx:idx+3].isupper():
#                 #         amino_counter += 1
#                 #         idx +=3
#                 #     elif  gene_seq[idx].islower():    #if its lower letter move forward
#                 #         idx+=1
#                 #     else:
#                 #         raise("somthig wrong with orf") 

#         # add site to list
#         list_sites_with_genes.append(site_with_changes)

# # keep sites wthin CDS only(remove only 4 sites)
# list_sites_with_genes = [site for site in list_sites_with_genes if 'Genes' in site.keys()]

add genes names

In [ ]:
# gene_dict_file = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/refseq_curated_ZNF_genes_name_dcit.txt"

# with open(gene_dict_file,"r") as genes_names_Fd:
#     DictReader_obj = csv.DictReader(genes_names_Fd, delimiter="\t")
#     genes_names_dict = {line["#name"]: line["name2"]  for line in DictReader_obj}

# for site in list_sites_with_genes:
#     site["gene_name"] = set([])
#     for gene_id in site["Genes"]:
#         site["gene_name"].add(genes_names_dict[gene_id])
# for site in list_sites_with_genes:
#     site["gene_name"]=list(site["gene_name"])
#     if len(site["gene_name"]) > 1:
#         if "-" in site["gene_name"][0]:
#             site["gene_name"] = site["gene_name"][1]
#         else:
#             site["gene_name"] = site["gene_name"][0]
# # make data readable
# for site in list_sites_with_genes:
#     site["Genes"]= ";".join(site["Genes"])
#     site["gene_name"]= "".join(site["gene_name"])
#     site['Amino_change'] = ";".join(site['Amino_change'])